In [1]:
import allel
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

def vcf_to_excel(vcf_path, excel_path, convert_genotypes=False, split_multiallelic=False):
    # Read VCF and create a dictionary
    vcf_df = vcf_to_df(vcf_path)
    samples = vcf_df.columns[6:]

    # Create a DataFrame for variants
    if split_multiallelic:
        vcf_df = split_rows_with_multiple_alleles(vcf_df, samples)

    # Convert genotypes to 0/1/2
    if convert_genotypes:
        vcf_df = convert_genotype_to_alt_allele_count(vcf_df, samples)
    
    if excel_path:
        # Write to Excel
        vcf_df.to_excel(excel_path, index=False)

def vcf_to_df(vcf_path, seg=True):
    
    vcf_dict = allel.read_vcf(vcf_path, fields='*')
    contig = vcf_dict['variants/CHROM'] 
    pos = vcf_dict['variants/POS']
    filter_pass = vcf_dict['variants/FILTER_PASS']
    ref = vcf_dict['variants/REF']
    alt = np.apply_along_axis(lambda x: ','.join(x[x != '']), 1, vcf_dict['variants/ALT'])
    ann = vcf_dict['variants/ANN']

    geno = allel.GenotypeArray(vcf_dict['calldata/GT'])

    if seg:
        ac = geno.count_alleles(max_allele=3)
        mask_seg = ac.is_segregating()
        geno = geno.compress(mask_seg, axis=0)
        contig = contig[mask_seg]
        pos = pos[mask_seg]
        filter_pass = filter_pass[mask_seg]
        ref = ref[mask_seg]
        alt = alt[mask_seg]
        ann = ann[mask_seg]

    ### make pd dataframe version of vcf
    vcf_df = pd.DataFrame({'CHROM': contig, 'POS': pos, 'FILTER_PASS': filter_pass, 'REF': ref, 'ALT': alt, 'ANN': ann})
    # make pd dataframe version of genotypes
    geno_df = pd.DataFrame(geno.to_gt().astype(str), columns=vcf_dict['samples'])
    vcf = pd.concat([vcf_df, geno_df], axis=1)
    return vcf

def split_rows_with_multiple_alleles(df, samples):
    # Create an empty list to store the new rows
    new_rows = []
    # Iterate through each row
    for index, row in df.iterrows():
        alt_alleles = row['ALT'].split(',')
        # Check if there are multiple alleles in the ALT field
        if len(alt_alleles) > 1:
            for allele_num, allele in enumerate(alt_alleles):
                # Create a new row for each allele
                new_row = row.copy()
                new_row['ALT'] = allele
                # Update genotype fields
                for col in samples:
                    genotype = row[col]
                    # Split the genotype and process it
                    if genotype != './.':
                        gt_alleles = genotype.split('/')
                        new_gt = ['0' if (int(gt) != allele_num + 1 and gt != '0') else gt for gt in gt_alleles]
                        new_row[col] = '/'.join(new_gt)
                new_rows.append(new_row)
        else:
            new_rows.append(row)
    
    # Create a new DataFrame from the new rows
    new_df = pd.DataFrame(new_rows).reset_index(drop=True)
    return new_df

def convert_genotype_to_alt_allele_count(df, samples):
    nalt_df = df.copy()
    # Iterate through each row
    for index, row in df.iterrows():
        # Update genotype fields
        for col in samples:
                genotype = row[col]
                if genotype != './.':
                    # Split the genotype and count non-zero alleles
                    alleles = genotype.split('/')
                    alt_allele_count = sum([1 for allele in alleles if allele != '0'])
                    nalt_df.at[index, col] = alt_allele_count
                else:
                    nalt_df.at[index, col] = np.nan

    return nalt_df

In [2]:
dataset = "ag-vampir-002"
wkdir = "../.."

In [3]:
# Parameters
dataset = "ag-vampir-002"
wkdir = "/home/snagi/lstm_projects/ampseq-agvampir002"


# SNP data

In this notebook, we display the variant calling and annotation results as a pandas dataframe and save it as an excel spreadsheet for the user to explore or analyse further. If the DataFrame is too large to display, please use the .xlsx file. 

#### Target SNP data

In [4]:
vcf_df = vcf_to_excel(
    vcf_path=f"{wkdir}/results/vcfs/targets/{dataset}.annot.vcf",
    excel_path=f"{wkdir}/results/vcfs/targets/{dataset}-snps.xlsx",
    convert_genotypes=True,
    split_multiallelic=True,
)
pd.set_option("display.max_rows", 1000, "display.max_columns", 1000)
vcf_df

---

In [5]:
from IPython.display import display, Markdown
display(Markdown(f'<a href="{wkdir}/results/vcfs/targets/{dataset}-snps.xlsx">Target SNP data (.xlsx)</a>'))
display(Markdown(f'<a href="{wkdir}/results/vcfs/targets/{dataset}.annot.vcf">Target SNP data (.vcf)</a>'))

<a href="/home/snagi/lstm_projects/ampseq-agvampir002/results/vcfs/targets/ag-vampir-002-snps.xlsx">Target SNP data (.xlsx)</a>

<a href="/home/snagi/lstm_projects/ampseq-agvampir002/results/vcfs/targets/ag-vampir-002.annot.vcf">Target SNP data (.vcf)</a>

#### Whole-amplicon SNP data

For times sake, we do not write out whole-amplicon .xlsx by default - it is slow. 

In [6]:
vcf_df = vcf_to_excel(
    vcf_path=f"{wkdir}/results/vcfs/amplicons/{dataset}.annot.vcf",
    excel_path=f"{wkdir}/results/vcfs/amplicons/{dataset}-snps.xlsx",
    convert_genotypes=True,
    split_multiallelic=True
)
vcf_df

---

In [7]:
display(Markdown(f'<a href="{wkdir}/results/vcfs/amplicons/{dataset}-snps.xlsx">Whole amplicon SNP data (.xlsx)</a>'))
display(Markdown(f'<a href="{wkdir}/results/vcfs/amplicons/{dataset}.annot.vcf">Whole amplicon SNP data (.vcf)</a>'))

<a href="/home/snagi/lstm_projects/ampseq-agvampir002/results/vcfs/amplicons/ag-vampir-002-snps.xlsx">Whole amplicon SNP data (.xlsx)</a>

<a href="/home/snagi/lstm_projects/ampseq-agvampir002/results/vcfs/amplicons/ag-vampir-002.annot.vcf">Whole amplicon SNP data (.vcf)</a>